# Experimento do Efeito Compton 

In [1]:
from scipy.constants import h, m_e, c
import pandas as pd
import math

<p align="center">
  <img src="/home/gabriel/Documentos/Física Experimental Avançada II/Electromagnetic_spectrum.svg" alt="Descrição da Imagem">
</p>

In [59]:
class ComptonScattering:
    """
    Class for handling Compton scattering experiments.

    Attributes:
    - theta (float): Scattering angle in radians.
    - U (float): Accelerating voltage in kilovolts.
    - I (float): Beam current in milliamperes.
    - delta_T (float): Time interval in seconds.
    - R_0 (float): Count rate without the copper filter.
    - R_1 (float): Count rate with the copper filter in front of the aluminum body.
    - R_2 (float): Count rate with the copper filter behind the aluminum body.
    - R (float): Background count rate.
    - theoretical_result (float): Theoretical wavelength shift.
    - experimental_result (float): Experimental wavelength shift.
    - error_experimental_result (float): Error associated with the experimental wavelength shift.

    Methods:
    - get_nm(value): Convert a value from meters to nanometers.
    - theoretical_wavelength_shift(): Calculate the theoretical wavelength shift due to scattering.
    - experimental_wavelength_shift(lambda_0=100e-12, a=7.6, n=2.75): Calculate the experimental wavelength shift.
    - experimental_wavelength_error(lambda_0=100e-12, a=7.6, n=2.75): Calculate the error associated with the experimental wavelength shift.
    """
    def __init__(self, data_series):
        """
        Initialize a ComptonScattering object with data from a pandas Series.

        Parameters:
        data_series (pandas.Series): A pandas Series containing data for Compton scattering experiments.
            - 'theta [rad]': Scattering angle in radians.
            - 'U [kV]': Accelerating voltage in kilovolts.
            - 'I [mA]': Beam current in milliamperes.
            - 'delta_T [s]': Time interval in seconds.
            - 'R_0 [s^-1]': Count rate without the copper filter.
            - 'R_1 [s^-1]': Count rate with the copper filter in front of the aluminum body.
            - 'R_2 [s^-1]': Count rate with the copper filter behind the aluminum body.
            - 'R [s^-1]': Background count rate.

        Attributes:
        theta (float): Scattering angle in radians.
        U (float): Accelerating voltage in kilovolts.
        I (float): Beam current in milliamperes.
        delta_T (float): Time interval in seconds.
        R_0 (float): Count rate without the copper filter.
        R_1 (float): Count rate with the copper filter in front of the aluminum body.
        R_2 (float): Count rate with the copper filter behind the aluminum body.
        R (float): Background count rate.
        theoretical_result (float): Theoretical wavelength shift.
        experimental_result (float): Experimental wavelength shift.
        error_experimental_result (float): Error associated with the experimental wavelength shift.
        """
        self.theta = data_series['theta [rad]']
        self.U = data_series['U [kV]']
        self.I = data_series['I [ma]']
        self.delta_T = data_series['delta_T [s]']
        self.R_0 = data_series['R_0 [s^-1]']
        self.R_1 = data_series['R_1 [s^-1]']
        self.R_2 = data_series['R_2 [s^-1]']
        self.R = data_series['R [s^-1]']

        self.theoretical_result = self.theoretical_wavelength_shift()
        self.experimental_result = self.experimental_wavelength_shift()
        self.error_experimental_result = self.experimental_wavelength_error()

    def get_nm(self, value):
        """
        Convert a value from meters to nanometers.

        Parameters:
        value (float): The value in meters.

        Returns:
        float: The value converted to nanometers.
        """
        return value * 1e9

    def theoretical_wavelength_shift(self):
        """
        Calculate the theoretical wavelength shift due to scattering.

        Parameters:
        theta (float): The scattering angle in radians.

        Returns:
        float: The theoretical wavelength shift.

        Constants:
        - h (float): Planck's constant (m^2 kg / s).
        - m_e (float): Electron rest mass (kg).
        - c (float): Speed of light in vacuum (m/s).
        """
        # Compton wavelength
        lambda_c = h / (m_e * c)

        # Wavelength shift formula
        delta_lambda_theoretical = lambda_c * (1 - math.cos(self.theta))

        return delta_lambda_theoretical
    
    def experimental_wavelength_shift(self, lambda_0=100e-12, a=7.6, n=2.75):
        """
        Calculate the experimental wavelength shift due to scattering based on the arrangement developed by R. W. Pohl.

        Parameters:
        R0 (float): Count rate obtained with the arrangement without the copper filter.
        R1 (float): Count rate obtained with the arrangement having the copper filter in front of the aluminum body.
        R2 (float): Count rate obtained with the arrangement having the copper filter behind the aluminum body.
        R (float): Background count rate.
        lambda_0 (float, optional): Wavelength without scattering in meters (default is 100 picometers).
        a (float, optional): Constant parameter (default is 7.6).
        n (float, optional): Constant parameter (default is 2.75).

        Returns:
        float: The experimental wavelength shift.

        Note:
        T1 (float): Coefficient of transmission (in copper) for the incident X-ray beam on the aluminum body.
        T2 (float): Coefficient of transmission (in copper) for the X-ray beam scattered by the aluminum body.
        """
        # Coefficients of transmission
        T_1 = (self.R_0 - self.R) / (self.R_1 - self.R)
        T_2 = (self.R_0 - self.R) / (self.R_2 - self.R)

        # Experimental wavelength shift formula
        term1 = ((1 / a) * math.log(T_2)) ** (1 / n)
        term2 = ((1 / a) * math.log(T_1)) ** (1 / n)

        delta_lambda_experimental = lambda_0 * (term1 - term2)

        return delta_lambda_experimental
    
    def experimental_wavelength_error(self, lambda_0=100e-12, a=7.6, n=2.75):
        """
        Calculate the error associated with the experimental Compton shift value based on the Pohl method.

        Parameters:
        R_0 (float): Count rate obtained with the arrangement without the copper filter.
        R_1 (float): Count rate obtained with the arrangement having the copper filter in front of the aluminum body.
        R_2 (float): Count rate obtained with the arrangement having the copper filter behind the aluminum body.
        R (float): Background count rate.
        delta_t (float): Time interval in seconds.
        lambda_0 (float, optional): Wavelength without scattering in meters (default is 100 picometers).
        a (float, optional): Constant parameter (default is 7.6).
        n (float, optional): Constant parameter (default is 2.75).

        Returns:
        float: The error associated with the experimental wavelength shift.
        """
        T_1 = (self.R_0 - self.R) / (self.R_1 - self.R)
        T_2 = (self.R_0 - self.R) / (self.R_2 - self.R)
        log_term_1 = math.log(T_1)
        log_term_2 = math.log(T_2)
        
        # Inside the square root
        A = self.R_1 / (((log_term_1 ** (7 / 11)) * (self.R_1 - self.R)) ** 2)
        B = self.R_2 / (((log_term_2 ** (7 / 11)) * (self.R_2 - self.R)) ** 2)
        C = ((((1 / (log_term_2 ** (7 / 11))) - (1 / (log_term_1 ** (7 / 11)))) * (1 / (self.R_0 - self.R))) ** 2) * self.R_0  
        D = ((((((self.R_0 - self.R_2) / (self.R_2 - self.R)) / (log_term_2 ** (7 / 11))) - (((self.R_0 - self.R_1) / (self.R_1 - self.R)) / (log_term_1 ** (7 / 11)))) * (1 / (self.R_0 - self.R))) ** 2) * self.R
        
        error = math.sqrt(((1 / self.delta_T) * (A + B + C + D))) *  (lambda_0 / n) * ((1 / a) ** (1 / n))

        return error

# Load data

In [3]:
data_series = pd.Series(
    {
        'theta [rad]' : math.radians(143.8),
        'U [kV]' : 30.,
        'I [ma]' : 1.,
        'delta_T [s]' : 300.,
        'R_0 [s^-1]' : 19.16,
        'R_1 [s^-1]' : 2.497,
        'R_2 [s^-1]' : 1.487,
        'R [s^-1]' : .300
    }
)

data_series.info()

<class 'pandas.core.series.Series'>
Index: 8 entries, theta [rad] to R [s^-1]
Series name: None
Non-Null Count  Dtype  
--------------  -----  
8 non-null      float64
dtypes: float64(1)
memory usage: 128.0+ bytes


In [4]:
data_series

theta [rad]      2.509783
U [kV]          30.000000
I [ma]           1.000000
delta_T [s]    300.000000
R_0 [s^-1]      19.160000
R_1 [s^-1]       2.497000
R_2 [s^-1]       1.487000
R [s^-1]         0.300000
dtype: float64

# Results

In [60]:
res = ComptonScattering(data_series)

In [67]:
print(f'Theoretical value of the Compton scattering: {res.theoretical_result * 10e11} pm')
print(f'Experimental value of the Compton scattering: {res.experimental_result * 10e11} pm')
print(f'Compton displacement experimental error value: {res.error_experimental_result * 10e11} pm')

Theoretical value of the Compton scattering: 4.384246306248503 pm
Experimental value of the Compton scattering: 6.058667087747161 pm
Compton displacement experimental error value: 0.7051911852871587 pm


In [68]:
data_series_2 = pd.Series(
    {
        'theta [rad]' : math.radians(143.5),
        'U [kV]' : 30.,
        'I [ma]' : 1.,
        'delta_T [s]' : 600.,
        'R_0 [s^-1]' : 18.15,
        'R_1 [s^-1]' : 2.223,
        'R_2 [s^-1]' : 1.613,
        'R [s^-1]' : .300
    }
)

data_series_2.info()

<class 'pandas.core.series.Series'>
Index: 8 entries, theta [rad] to R [s^-1]
Series name: None
Non-Null Count  Dtype  
--------------  -----  
8 non-null      float64
dtypes: float64(1)
memory usage: 128.0+ bytes


# Djalma data and results

In [69]:
res2 = ComptonScattering(data_series_2)

In [70]:
print(f'Theoretical value of the Compton scattering: {res2.theoretical_result * 10e11} pm')
print(f'Experimental value of the Compton scattering: {res2.experimental_result * 10e11} pm')
print(f'Compton displacement experimental error value: {res2.error_experimental_result * 10e11} pm')

Theoretical value of the Compton scattering: 4.376716370034293 pm
Experimental value of the Compton scattering: 3.7870149327879803 pm
Compton displacement experimental error value: 0.5002318461962434 pm
